[View in Colaboratory](https://colab.research.google.com/github/nautilusPrime/cifar10-image-classifier/blob/master/cifar10_tl.ipynb)

In [1]:
import keras
from keras.datasets import cifar10
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from skimage.transform import resize
import numpy as np
import os

Using TensorFlow backend.


In [0]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_trained_model.h5'

In [0]:
def resize_and_subsample(X_train, y_train, X_test, y_test):
    
    print "Resizing training samples..."
    # resized x_train
    resized = []
    for image in X_train[:10000]:
        resized += [resize(image, (139, 139), mode='constant')]
        
    np.savez(open(save_dir+"/X_train_resized_2000.npz", 'w'), data=resized)
    
    print "Resizing test samples..."
    # resized x_test
    resized = []
    for image in X_test[:2000]:
        resized += [resize(image, (139, 139), mode='constant')]
        
    print "Saving the resized and subsampled files..."
    np.savez(open(save_dir+"/X_test_resized_200.npz", 'w'), data=resized)
    np.savez(open(save_dir+"/y_test_200.npz", 'w'), data=y_test[:2000])
    np.savez(open(save_dir+"/y_train_2000.npz", 'w'), data=y_train[:10000])
    
    return (X_train, y_train), (X_test, y_test)

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
!rm -rf $save_dir

In [0]:
def load_dataset():
    
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.exists(save_dir):
      os.makedirs(save_dir)
    X_train_path = save_dir+"/X_train_resized_2000.npz"
    X_test_path = save_dir+"/X_test_resized_200.npz"
    y_train_path = save_dir+"/y_train_2000.npz"
    y_test_path = save_dir+"/y_test_200.npz"
    
    if os.path.exists(X_train_path) and os.path.exists(X_test_path) and os.path.exists(y_train_path) and os.path.exists(y_test_path):
        print "Train and test samples have already been resized and subsampled."
    else:
        print "Train and test samples need to be resized and subsampled..."
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()
        (X_train, y_train), (X_test, y_test) = resize_and_subsample(X_train, y_train, X_test, y_test)
        print "Done."
        
    print "Loading the files from disk..."
    # loading data
    X_train = np.load(open(save_dir+"/X_train_resized_2000.npz", 'r'))
    X_train = X_train['data']
    
    X_test = np.load(open(save_dir+"/X_test_resized_200.npz", 'r'))
    X_test = X_test['data']
    
    y_train = np.load(open(save_dir+"/y_train_2000.npz", 'r'))
    y_train = y_train['data']
    y_test = np.load(open(save_dir+"/y_test_200.npz", 'r'))
    y_test = y_test['data']
    
    print "Done."
    
    return (X_train, y_train), (X_test, y_test)

In [5]:
# load the dataset
(X_train, y_train), (X_test, y_test) = load_dataset()

Train and test samples have already been resized and subsampled.
Loading the files from disk...
Done.


In [6]:
print X_train.shape, y_train.shape, X_test.shape, y_test.shape

(10000, 139, 139, 3) (10000, 1) (2000, 139, 139, 3) (2000, 1)


In [0]:
#preprocess
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

### Bottleneck feature creation

In [0]:
model = VGG16(include_top=False, input_shape=X_train.shape[1:], classes=num_classes, weights='imagenet')

In [11]:
bottleneck_features_train = model.predict(X_train, verbose=1, batch_size=batch_size)

10000/10000 [==============================] - 47s 5ms/step


In [0]:
np.save(open(save_dir+'/bottleneck_features_train.npy', 'w'), bottleneck_features_train)

In [0]:
from google.colab import files 
files.download(save_dir+'/bottleneck_features_train.npy')

In [13]:
bottleneck_features_val = model.predict(X_test, verbose=1, batch_size=batch_size)

2000/2000 [==============================] - 9s 4ms/step


In [0]:
np.save(open(save_dir+'/bottleneck_features_val.npy', 'w'), bottleneck_features_val)

In [0]:
from google.colab import files 
files.download(save_dir+'/bottleneck_features_val.npy')

In [0]:
train_data = np.load(open(save_dir+'/bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = y_train


In [16]:
train_data.shape

(10000, 4, 4, 512)

In [0]:
validation_data = np.load(open(save_dir+'/bottleneck_features_val.npy'))
validation_labels = y_test

In [0]:
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(train_data, train_labels,
          epochs=100,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

Train on 10000 samples, validate on 2000 samples
Epoch 1/100
10000/10000 [==============================] - 4s 404us/step - loss: 1.7912 - acc: 0.3663 - val_loss: 1.2650 - val_acc: 0.5725
Epoch 2/100
10000/10000 [==============================] - 3s 348us/step - loss: 1.3942 - acc: 0.4964 - val_loss: 1.1620 - val_acc: 0.6085
Epoch 3/100
10000/10000 [==============================] - 4s 350us/step - loss: 1.2530 - acc: 0.5532 - val_loss: 1.0705 - val_acc: 0.6225
Epoch 4/100
10000/10000 [==============================] - 3s 349us/step - loss: 1.1719 - acc: 0.5848 - val_loss: 1.0258 - val_acc: 0.6385
Epoch 5/100
 3200/10000 [========>.....................] - ETA: 2s - loss: 1.1447 - acc: 0.5903

10000/10000 [==============================] - 4s 352us/step - loss: 1.1235 - acc: 0.5976 - val_loss: 1.0349 - val_acc: 0.6340
Epoch 6/100
10000/10000 [==============================] - 4s 352us/step - loss: 1.0828 - acc: 0.6193 - val_loss: 1.1356 - val_acc: 0.6190
Epoch 7/100
10000/10000 [==============================] - 4s 357us/step - loss: 1.0418 - acc: 0.6460 - val_loss: 0.9836 - val_acc: 0.6555
Epoch 8/100
10000/10000 [==============================] - 3s 347us/step - loss: 0.9980 - acc: 0.6506 - val_loss: 0.9069 - val_acc: 0.6965
Epoch 9/100
 7744/10000 [======================>.......] - ETA: 0s - loss: 0.9667 - acc: 0.6626

10000/10000 [==============================] - 4s 353us/step - loss: 0.9722 - acc: 0.6609 - val_loss: 0.9050 - val_acc: 0.7090
Epoch 10/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.9386 - acc: 0.6719 - val_loss: 0.9997 - val_acc: 0.6695
Epoch 11/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.9152 - acc: 0.6813 - val_loss: 0.8994 - val_acc: 0.7145
Epoch 12/100
10000/10000 [==============================] - 3s 350us/step - loss: 0.8898 - acc: 0.6937 - val_loss: 1.1068 - val_acc: 0.6695
Epoch 13/100
 8064/10000 [=======================>......] - ETA: 0s - loss: 0.8906 - acc: 0.6906

10000/10000 [==============================] - 4s 350us/step - loss: 0.8881 - acc: 0.6904 - val_loss: 0.9844 - val_acc: 0.7060
Epoch 14/100
10000/10000 [==============================] - 4s 353us/step - loss: 0.8720 - acc: 0.7012 - val_loss: 0.9704 - val_acc: 0.6875
Epoch 15/100
10000/10000 [==============================] - 4s 361us/step - loss: 0.8464 - acc: 0.7084 - val_loss: 0.9344 - val_acc: 0.7045
Epoch 16/100
10000/10000 [==============================] - 4s 354us/step - loss: 0.8282 - acc: 0.7177 - val_loss: 0.9802 - val_acc: 0.7115
Epoch 17/100
 7968/10000 [======================>.......] - ETA: 0s - loss: 0.8227 - acc: 0.7175

10000/10000 [==============================] - 4s 351us/step - loss: 0.8279 - acc: 0.7160 - val_loss: 1.0728 - val_acc: 0.6670
Epoch 18/100
10000/10000 [==============================] - 4s 350us/step - loss: 0.8009 - acc: 0.7265 - val_loss: 1.0736 - val_acc: 0.6715
Epoch 19/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.8006 - acc: 0.7230 - val_loss: 0.9857 - val_acc: 0.7035
Epoch 20/100
10000/10000 [==============================] - 3s 348us/step - loss: 0.7916 - acc: 0.7274 - val_loss: 0.9707 - val_acc: 0.7175
Epoch 21/100
 8480/10000 [========================>.....] - ETA: 0s - loss: 0.7697 - acc: 0.7328

...

10000/10000 [==============================] - 3s 349us/step - loss: 0.6063 - acc: 0.8050 - val_loss: 1.5544 - val_acc: 0.6970
Epoch 82/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.6271 - acc: 0.8027 - val_loss: 1.6726 - val_acc: 0.6940
Epoch 83/100
10000/10000 [==============================] - 4s 350us/step - loss: 0.6239 - acc: 0.8046 - val_loss: 1.5835 - val_acc: 0.6865
Epoch 84/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.6176 - acc: 0.8086 - val_loss: 1.6872 - val_acc: 0.7050
Epoch 85/100
 8416/10000 [========================>.....] - ETA: 0s - loss: 0.6234 - acc: 0.8058

10000/10000 [==============================] - 3s 348us/step - loss: 0.6317 - acc: 0.8043 - val_loss: 1.6263 - val_acc: 0.6885
Epoch 86/100
10000/10000 [==============================] - 3s 349us/step - loss: 0.6412 - acc: 0.8042 - val_loss: 1.5858 - val_acc: 0.6980
Epoch 87/100
10000/10000 [==============================] - 3s 347us/step - loss: 0.6354 - acc: 0.8022 - val_loss: 1.7601 - val_acc: 0.6890
Epoch 88/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.6034 - acc: 0.8154 - val_loss: 1.6376 - val_acc: 0.7120
Epoch 89/100
 8384/10000 [========================>.....] - ETA: 0s - loss: 0.6052 - acc: 0.8099

10000/10000 [==============================] - 3s 348us/step - loss: 0.6133 - acc: 0.8062 - val_loss: 1.9031 - val_acc: 0.6860
Epoch 90/100
10000/10000 [==============================] - 3s 346us/step - loss: 0.6193 - acc: 0.8111 - val_loss: 1.6417 - val_acc: 0.6925
Epoch 91/100
10000/10000 [==============================] - 4s 352us/step - loss: 0.6136 - acc: 0.8090 - val_loss: 1.6170 - val_acc: 0.6930
Epoch 92/100
10000/10000 [==============================] - 3s 348us/step - loss: 0.6289 - acc: 0.8034 - val_loss: 1.7133 - val_acc: 0.7070
Epoch 93/100
 8576/10000 [========================>.....] - ETA: 0s - loss: 0.6095 - acc: 0.8094

10000/10000 [==============================] - 4s 350us/step - loss: 0.6120 - acc: 0.8078 - val_loss: 1.7146 - val_acc: 0.6990
Epoch 94/100
10000/10000 [==============================] - 4s 351us/step - loss: 0.6212 - acc: 0.8095 - val_loss: 1.6811 - val_acc: 0.6945
Epoch 95/100
10000/10000 [==============================] - 3s 347us/step - loss: 0.6152 - acc: 0.8101 - val_loss: 1.6652 - val_acc: 0.6935
Epoch 96/100
10000/10000 [==============================] - 3s 347us/step - loss: 0.6394 - acc: 0.8015 - val_loss: 1.6913 - val_acc: 0.7010
Epoch 97/100
 8608/10000 [========================>.....] - ETA: 0s - loss: 0.6258 - acc: 0.8072

10000/10000 [==============================] - 3s 347us/step - loss: 0.6159 - acc: 0.8091 - val_loss: 1.7389 - val_acc: 0.7050
Epoch 98/100
10000/10000 [==============================] - 3s 349us/step - loss: 0.6601 - acc: 0.8027 - val_loss: 1.7682 - val_acc: 0.6900
Epoch 99/100
10000/10000 [==============================] - 4s 358us/step - loss: 0.6484 - acc: 0.8064 - val_loss: 1.6784 - val_acc: 0.6980
Epoch 100/100
10000/10000 [==============================] - 4s 356us/step - loss: 0.6115 - acc: 0.8075 - val_loss: 1.6642 - val_acc: 0.6985


In [0]:
model.save_weights(save_dir+'/bottleneck_fc_model.h5')